In [9]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import certifi
import urllib3
import warnings
warnings.filterwarnings("ignore")
import os
import urllib3
from bs4 import BeautifulSoup
from requests.packages.urllib3.exceptions import InsecureRequestWarning
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_core.documents import Document
import os
import certifi
import time
os.environ['REQUESTS_CA_BUNDLE'] = certifi.where()

from langchain.document_loaders import UnstructuredURLLoader

### **Data Collecations**

In [ ]:
def get_all_links(url):

# Suppress only the InsecureRequestWarning
    requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

    http = urllib3.PoolManager(cert_reqs='CERT_NONE')
    response = http.request('GET', url)

    # Check if the request was successful
    if response.status != 200:  # تأكد من حالة الاستجابة
        print(f"Failed to retrieve the page. Status code: {response.status}")
        return []

    # Parse the HTML content of the page
    soup = BeautifulSoup(response.data.decode('utf-8'), 'html.parser')  

    # Find all links
    links = []
    for link in soup.find_all('a'):
        href = link.get('href')
        if href:
            links.append(href)

    return links

# Example usage
url = 'https://neoma-bs.com/'  
all_links = get_all_links(url)
print("All links found:")
for link in all_links:
    print(link)


In [11]:
# Store link without filteration into txt file

with open("URL.txt",'w')as f:
    for link in all_links:
        f.write(link + "\n")
        


### **Data Preprocessing**

In [13]:
# Remove a extenations from link (.pdf,.docs)

link_without_exe = []
def link_exe(linkes):
    for link in linkes:
        exe,_ =  os.path.splitext(link)
        link_without_exe.append(exe)
    return link_without_exe


In [ ]:
linkes = link_exe(all_links)

In [15]:
# Check the pattern if exist on the link 

final_link = []
pattern = "https://neoma"

for link in linkes:
    if pattern in link:
        final_link.append(link)

In [17]:
# removes the repetition

final_link = set(final_link)

In [10]:
# store link into txt file

with open("FinalURL.txt",'w')as f:
    for link in final_link:
        f.write(link + "\n")
        

In [19]:
# Load Web page content
rs= []
for link in final_link:
    r=requests.get(link, verify=False)
    rs.append(r.content)

In [54]:
# Load page content into 

# Iterate through each line in the source 'rs'
data_collecation = []

for line in rs:
# create the HTML file and write the decoded line into it
    with open("./data_html.html","w")as f:
        f.write(line.decode())
# Load the content of the newly created HTML file using UnstructuredHTMLLoader
    loadder = UnstructuredHTMLLoader("./data_html.html")
    data = loadder.load()

    data_collecation.append(data)


In [ ]:
documents = []
# Iterate through each collection of documents in data_collecation
for line in data_collecation:
    # Iterate through each Document in the current collection (line)
    for doc in line:
        documents.append(Document(doc.page_content))

In [59]:
# Using HuHuggingFaceEmbeddings mode;
embedding_function = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

# Stroe Documents and embeddings
vectorstore = Chroma.from_documents(documents, embedding_function,persist_directory="./")

# Retrive data using cosine similarity
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 6}
)
# Store data into local disk
vectorstore.persist()

### **Test Query**

In [68]:
query = "What's the program?"

result = retriever.get_relevant_documents(query)[0]

print(result.page_content)

open alert

INTAKE 2025 I Applications are open to join our programme

Apply now

MSc Wine & Gastronomy

MASTER THE ART OF WINE & GASTRONOMY MANAGEMENT

Brochure

Apply

Home

Choose your programme

MSc Wine & Gastronomy

Programme overview

Curriculum

Careers

Admissions

2-Year MSc Option

Testimonials

Student life
                            
		    			
				Student life
			
	    			
				Financing
			
	    			
				Student support

The MSc Wine & Gastronomy offers an insider look at the marketing and management challenges facing food, wine and spirits professionals.

Combining academic lectures and immersive experiences, the programme offers direct contact with companies and top managers from the food & beverage industry.

Your MSc in 2 years!

If you hold a 3-year Bachelor’s degree, you may complete this MSc in two years by joining our 12-month International Pre-Master programme. For more information on this option, see the “2-Year MSc Option” tab!

Programme strengths

Master the u

## **Thank you Ahmed for your support and cooperation**